https://groups.google.com/g/the-art-of-multiprocessor-programming-2nd-edition

[](/_notes/CS/Concurrency.md)
https://lobste.rs/s/va2mvi/without_futex_it_s_futile


Lamport book  https://lamport.azurewebsites.net/tla/book-21-07-04.pdf   https://lamport.azurewebsites.net/tla/high-level-view.html
https://eatonphil.com/bookclub.html

Lynch distributed algorithms

https://cs.brown.edu/courses/cs176/ 

https://harmony.cs.cornell.edu/book/ https://harmony.cs.cornell.edu/ https://github.com/harmonylang/harmony

https://github.com/fizzbee-io/fizzbee
alloy https://practicalalloy.github.io/chapters/behavioral-modeling/index.html
ivy?
quint https://quint-lang.org/


# chapter 1 intro
Alice and Bob cats and Dogs https://lamport.azurewebsites.net/pubs/solved-and-unsolved.pdf

Prdoucer consumer

Dining philosphers
https://cocotec.io/fdr/ 
```
N = 8

PHILNAMES = {0..N-1}
FORKNAMES = {0..N-1}

channel thinks, sits, eats, getsup : PHILNAMES

channel picks, putsdown : PHILNAMES.FORKNAMES

-- A philosopher thinks, sits down, picks up two forks,
-- eats, puts down forks and gets up, in an unending
-- cycle.

PHIL(i) =
  thinks.i -> sits!i -> picks!i!i ->
  picks!i!((i+1)%N) -> eats!i -> putsdown!i!((i+1)%N) ->
  putsdown!i!i -> getsup!i -> PHIL(i)

assert PHIL(0) :[deadlock free [F]]
```


A shared counter



In [ ]:
%%file /tmp/configfile.cfg
SPECIFICATION Spec

INVARIANT Inv1, Inv2
PROPERTY Prop1

In [ ]:
%%file /tmp/alias.tla
---- MODULE aliases ----
EXTENDS Integers

VARIABLE x
Init ==
  x = 0

Next == x' = x + 1
Inv == x < 10
Spec == Init /\ [][Next]_x

Alias ==
  [x |-> x,
   nextx |-> x',
   incx |-> x + 1]
=====

In [15]:
%%file /tmp/hello.java

void main() {
    System.out.println("Hello, World!");
}

Writing /tmp/hello.java


Amdahl's law

fraction that can be aprallized p
1 - p + p/n

In [22]:
! java --enable-preview --source 21 /tmp/hello.java

Note: /tmp/hello.java uses preview features of Java SE 21.
Note: Recompile with -Xlint:preview for details.
Hello, World!


https://docs.oracle.com/javase/8/docs/api/java/util/concurrent/package-summary.html

In [ ]:
%%file /tmp/primePrint.java

   void main() {
      int i = ThreadID.get(); // thread IDs are in {0..9}
      long block = power(10, 9);
      for ( long j = (i * block) + 1; j <= (i + 1) * block; j++) {
   if (isPrime(j))
      print(j);
   }
   }



Overwriting /tmp/primePrint.java


In [23]:
! java --enable-preview --source 21 /tmp/primePrint.java 

/tmp/primePrint.java:4: error: cannot find symbol
      int i = ThreadID.get(); // thread IDs are in {0..9}
              ^
  symbol:   variable ThreadID
  location: class primePrint
/tmp/primePrint.java:5: error: cannot find symbol
      long block = power(10, 9);
                   ^
  symbol:   method power(int,int)
  location: class primePrint
/tmp/primePrint.java:7: error: cannot find symbol
   if (isPrime(j))
       ^
  symbol:   method isPrime(long)
  location: class primePrint
/tmp/primePrint.java:8: error: cannot find symbol
      print(j);
      ^
  symbol:   method print(long)
  location: class primePrint
4 errors
error: compilation failed


# chapter 2 Mutual Exclusion
Bakery algorithm
Peterson Lock

https://en.wikipedia.org/wiki/Peterson%27s_algorithm

https://jamesrwilcox.com/SharedMem.html proof of peterson in coq
https://homes.cs.washington.edu/~mernst/pubs/verify-distsystem-pldi2015.pdf verdi

https://harmony.cs.cornell.edu/book/#sec-33

https://cs.brown.edu/~tbn/publications/peterson.pdf alloy peterson

https://forge-fm.org/

ivy?
Transitive clousre vs int parametrized.

https://stackoverflow.com/questions/11588514/a-tested-implementation-of-peterson-lock-algorithm
https://bartoszmilewski.com/2008/11/05/who-ordered-memory-fences-on-an-x86/

https://ocw.mit.edu/courses/6-172-performance-engineering-of-software-systems-fall-2018/resources/lecture-17-synchronization-without-locks/

mutual exclusion
no deadlock - if multuple try, at least one succeeds
no starvation - if attempt, must eventually succeed

Loom / shuttle https://github.com/awslabs/shuttle
https://matklad.github.io/2024/07/05/properly-testing-concurrent-data-structures.html https://news.ycombinator.com/item?id=40890035

https://github.com/dvyukov/relacy

https://plv.mpi-sws.org/genmc/ https://github.com/mpi-sws/genmc/ Generic model checker for concurrent C programs

https://github.com/microsoft/coyote/ C# https://microsoft.github.io/coyote/

https://arxiv.org/abs/2501.12618 Fray Java https://github.com/cmu-pasta/fray

https://p-org.github.io/P/whatisP/

## Spin

In [3]:
%%file /tmp/lock1.pml

bool flag[2];
byte ncrit;

active [2] proctype user()
{
	assert(_pid == 0 || _pid == 1);
again:
	flag[_pid] = 1;
	flag[1 - _pid] == 0;

	ncrit++;
	assert(ncrit == 1);	/* critical section */
	ncrit--;

	flag[_pid] = 0;
	goto again
}

Writing /tmp/lock1.pml


In [5]:
! spin -v /tmp/lock1.pml

  1:	proc  0 (user:1) /tmp/lock1.pml:7 (state 1)	[assert(((_pid==0)||(_pid==1)))]
  2:	proc  1 (user:1) /tmp/lock1.pml:7 (state 1)	[assert(((_pid==0)||(_pid==1)))]
  3:	proc  1 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
  4:	proc  1 (user:1) /tmp/lock1.pml:10 (state 3)	[((flag[(1-_pid)]==0))]
  5:	proc  0 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
  6:	proc  1 (user:1) /tmp/lock1.pml:12 (state 4)	[ncrit = (ncrit+1)]
  7:	proc  1 (user:1) /tmp/lock1.pml:13 (state 5)	[assert((ncrit==1))]
  8:	proc  1 (user:1) /tmp/lock1.pml:14 (state 6)	[ncrit = (ncrit-1)]
  9:	proc  1 (user:1) /tmp/lock1.pml:16 (state 7)	[flag[_pid] = 0]
 10:	proc  1 (user:1) /tmp/lock1.pml:17 (state 8)	[goto again]
 11:	proc  0 (user:1) /tmp/lock1.pml:10 (state 3)	[((flag[(1-_pid)]==0))]
 12:	proc  0 (user:1) /tmp/lock1.pml:12 (state 4)	[ncrit = (ncrit+1)]
 13:	proc  1 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
 14:	proc  0 (user:1) /tmp/lock1.pml:13 (state 5)	[assert((ncrit==1))]
 15:	pro

In [1]:
%%file /tmp/peterson.pml
// https://github.com/nimble-code/Spin/blob/master/Examples/peterson.pml
/* Peterson's solution to the mutual exclusion problem - 1981 */

bool turn, flag[2];
byte ncrit;

active [2] proctype user()
{
	assert(_pid == 0 || _pid == 1);
again:
	flag[_pid] = 1;
	turn = _pid;
	(flag[1 - _pid] == 0 || turn == 1 - _pid);

	ncrit++;
	assert(ncrit == 1);	/* critical section */
	ncrit--;

	flag[_pid] = 0;
	goto again
}

Writing /tmp/peterson.pml


## tla

In [17]:
%%file /tmp/Peterson.tla
---- MODULE Peterson ----
EXTENDS Integers
(*
--algorithm Peterson {
    variables flag = [i \in {0, 1} |-> FALSE],  turn = 0;
        \* Declares the global variables flag and turn and their initial values;
        \* flag is a 2-element array with initially flag[0] = flag[1] = FALSE.
    fair process (proc \in {0,1}) {
    \* Declares two processes with identifier self equal to 0 and 1.
    \* The keyword fair means that no process can stop forever if it can
    \* always take a step.
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a3:  turn := 1 - self ;   
    a4:  await (flag[1-self] = FALSE) \/ (turn = self); \* \/ is written || in C.
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
====

Overwriting /tmp/Peterson.tla


In [18]:
! java -cp ~/Downloads/tla2tools.jar pcal.trans /tmp/Peterson.tla && cat /tmp/Peterson.tla

pcal.trans Version 1.11 of 31 December 2020
Parsing completed.
Translation completed.
New file /tmp/Peterson.tla written.
New file /tmp/Peterson.cfg written.
---- MODULE Peterson ----
EXTENDS Integers
(*
--algorithm Peterson {
    variables flag = [i \in {0, 1} |-> FALSE],  turn = 0;
        \* Declares the global variables flag and turn and their initial values;
        \* flag is a 2-element array with initially flag[0] = flag[1] = FALSE.
    fair process (proc \in {0,1}) {
    \* Declares two processes with identifier self equal to 0 and 1.
    \* The keyword fair means that no process can stop forever if it can
    \* always take a step.
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a3:  turn := 1 - self ;   
    a4:  await (flag[1-self] = FALSE) \/ (turn = self); \* \/ is written || in C.
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
\* BEGIN TRANSLATION (chksum(pcal) =

In [19]:
! java -cp ~/Downloads/tla2tools.jar tlc2.TLC /tmp/Peterson.tla

TLC2 Version 2.19 of 08 August 2024 (rev: 5a47802)


(Use the -nowarning option to disable this warning.)
Running breadth-first search Model-Checking with fp 111 and seed -1918630056517602505 with 1 worker on 16 cores with 15736MB heap and 64MB offheap memory [pid: 3180596] (Linux 6.14.0-28-generic amd64, Ubuntu 21.0.8 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /tmp/Peterson.tla
Parsing file /tmp/Integers.tla
Parsing file /tmp/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module Peterson
Starting... (2025-09-03 09:28:41)
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2025-09-03 09:28:41.
Model checking completed. No error has been found.
  Estimates of the probability that TLC did not check all reachable states
  because two distinct states had the same fingerprint:
  calculated (optimistic):  val = 8.0E-17
77 states generated, 42 distinct states found, 0 states left on queue.
The depth of the complete state graph se

In [20]:
%%file /tmp/lock1.tla
---- MODULE lock1 ----
EXTENDS Integers
(*

--algorithm Lock1 {
    variables flag = [i \in {0, 1} |-> FALSE];
    fair process (proc \in {0,1}) {
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a4:  await (flag[1-self] = FALSE);
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
====



Writing /tmp/lock1.tla


In [21]:
! java -cp ~/Downloads/tla2tools.jar pcal.trans /tmp/lock1.tla && java -cp ~/Downloads/tla2tools.jar tlc2.TLC /tmp/lock1.tla

pcal.trans Version 1.11 of 31 December 2020
Parsing completed.
Translation completed.
New file /tmp/lock1.tla written.
New file /tmp/lock1.cfg written.
TLC2 Version 2.19 of 08 August 2024 (rev: 5a47802)
(Use the -nowarning option to disable this warning.)
Running breadth-first search Model-Checking with fp 77 and seed -3099753534938489767 with 1 worker on 16 cores with 15736MB heap and 64MB offheap memory [pid: 3187713] (Linux 6.14.0-28-generic amd64, Ubuntu 21.0.8 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /tmp/lock1.tla
Parsing file /tmp/Integers.tla
Parsing file /tmp/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module lock1
Starting... (2025-09-03 09:32:10)
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2025-09-03 09:32:10.
Error: Deadlock reached.
Error: The behavior up to this point is:
State 1: <Initial predicate>
/\ flag = (0 :> FALSE @@ 1 :> FALSE)
/\ pc =

### apalache
What if I could use apalache to literally export tla to smt.

https://apalache-mc.org/docs/apalache/example.html

check
simulate
trnaspile


In [14]:
%%file /tmp/y2k.tla
-------------------------------- MODULE y2k --------------------------------
EXTENDS Integers
 
CONSTANT
    \* @type: Int;
    BIRTH_YEAR, \* the year to start with, between 0 and 99
    \* @type: Int;
    LICENSE_AGE    \* the minimum age to obtain a license
         
ASSUME(BIRTH_YEAR \in 0..99)              
ASSUME(LICENSE_AGE \in 1..99)              
 
VARIABLE
    \* @type: Int;
    year,
    \* @type: Bool;
    hasLicense

Age == year - BIRTH_YEAR 

Init ==
    /\ year = BIRTH_YEAR
    /\ hasLicense = FALSE
    
NewYear ==
    /\ year' = (year + 1) % 100 \* the programmers decided to use two digits
    /\ UNCHANGED hasLicense
    
IssueLicense ==
    /\ Age >= LICENSE_AGE
    /\ hasLicense' = TRUE
    /\ UNCHANGED year
    
Next ==
    \/ NewYear
    \/ IssueLicense

\* The somewhat "obvious" invariant, which is violated    
Safety ==
    hasLicense => (Age >= LICENSE_AGE)
\* This is probably the only invariant we can formulate, usually, it is called TypeOK    
Inv ==
    /\ year \in 0..99
    /\ hasLicense \in BOOLEAN
===================

Overwriting /tmp/y2k.tla


In [34]:
%%file /tmp/HourClock.tla
---------------------- MODULE HourClock ----------------------
EXTENDS Naturals
VARIABLE 
    \* @type: Int;
    hr
Init  ==  hr \in (1 .. 12)
Next  ==  hr' = IF hr # 13 THEN hr + 1 ELSE 1
Inv == Init
==============================================================

Overwriting /tmp/HourClock.tla


In [35]:
! ~/Downloads/apalache/bin/apalache-mc check --length=10  --inv=Inv /tmp/HourClock.tla

# Usage statistics is OFF. We care about your privacy.
# If you want to help our project, consider enabling statistics with config --enable-stats=true.

Output directory: /home/philip/philzook58.github.io/_drafts/_apalache-out/HourClock.tla/2025-09-11T11-55-02_10050244532456205326
# APALACHE version: 0.50.3 | build: be35470                       I@11:55:02.490
Tuning: search.outputTraces=false                                 I@11:55:02.510
PASS #0: SanyParser                                               I@11:55:02.638
Parsing file /tmp/HourClock.tla
Parsing file /tmp/SANYPBHwvecSS3/Naturals.tla
PASS #1: TypeCheckerSnowcat                                       I@11:55:02.785
 > Running Snowcat .::.                                           I@11:55:02.785
 > Your types are purrfect!                                       I@11:55:02.846
 > All expressions are typed                                      I@11:55:02.846
PASS #2: ConfigurationPass                                        I@11:55

In [41]:
! ~/Downloads/apalache/bin/apalache-mc check --length=1 --smt-encoding=funArrays --debug --write-intermediate /tmp/HourClock.tla

# Tool home: /home/philip/Downloads/apalache/bin
# Package:   /home/philip/Downloads/apalache/bin/../lib/apalache.jar
# JVM args:  -Xmx4096m
# -Djava.io.tmpdir: /tmp/SANYDqFvTjI3F5
#
# Usage statistics is OFF. We care about your privacy.
# If you want to help our project, consider enabling statistics with config --enable-stats=true.

Output directory: /home/philip/philzook58.github.io/_drafts/_apalache-out/HourClock.tla/2025-09-11T12-06-51_16913358135050887983
# APALACHE version: 0.50.3 | build: be35470                       I@12:06:51.058
Tuning: search.outputTraces=false                                 I@12:06:51.074
PASS #0: SanyParser                                               I@12:06:51.213
Parsing file /tmp/HourClock.tla
Parsing file /tmp/SANYDqFvTjI3F5/Naturals.tla
PASS #1: TypeCheckerSnowcat                                       I@12:06:51.408
 > Running Snowcat .::.                                           I@12:06:51.408
 > Your types are purrfect!                         

## quint
npm i @informalsystems/quint -g

uses apalache. Can I dump apalache smt?

In [ ]:
%%file /tmp/test.qnt

var x: int
 
action init = x' = 1
 
action step = x' = x + 1


In [9]:
! quint

Quint REPL 0.27.0
Type ".exit" to exit, or ".help" for more information
>>> ^C


## alloy
https://github.com/AlloyTools/org.alloytools.alloy/blob/master/org.alloytools.alloy.extra/extra/models/examples/algorithms/peterson.als

In [5]:
%%file /tmp/lock.als
module examples/algorithms/peterson

open util/ordering[priority] as po
// open util/ordering[State] as so

sig pid {
}

sig priority {
}

fact {
  # priority = # pid + 1
}

Overwriting /tmp/lock.als


In [6]:
! cd /tmp && /home/philip/Downloads/alloy-6.2.0/bin/alloy exec /tmp/lock.als

Error
  0. The output directory /tmp/lock contains files. Delete them or use the -f option


In [7]:
%%file /tmp/file.als
sig Object {}
sig Dir in Object {}
sig File in Object {}

run example {}

Writing /tmp/file.als


In [8]:
! cd /tmp && /home/philip/Downloads/alloy-6.2.0/bin/alloy exec /tmp/file.als

00. run   example                      1/1     SAT


## python. 
Use yield or await?
hypotehsis? Take cue from rust yarn libs
https://hypothesis.readthedocs.io/en/latest/stateful.html
https://hypothesis.works/articles/how-not-to-die-hard-with-hypothesis/
https://hypothesis.works/articles/rule-based-stateful-testing/ 
Seems more painful than is worth.



esbmc actually supports python. Hmmm.


pcode execution with 


In [ ]:
from hypothesis.stateful import RuleBasedStateMachine, precondition, rule
class PC(Enum):
    Init = 0
    Next = 1
    Inc = 2
    CRIT = 3

class Peterson(RuleBasedStateMachine):
    pc = [PC.Init, PC.Init]
    flag = [False, False]
    turn = 0

    @precondition(lambda self: self.pc[0] == PC.Init and self.pc[1] == PC.Init)
    @rule()
    def set_flag(self):

In [ ]:
ProcID = kd.Enum("ProcID", ["P0", "P1"])
PC = kd.Enum("PC", ["Init", "Flag1", "Flag2", "Crit", "Exit"])
Spec = smt.Function("Spec", smt.ArraySort(ProcID, smt.BoolSort()), smt.ArraySort(ProcID, smt.BoolSort()))
Spec = kd.define("Spec", [flag, pc, flag1, pc1], 
    smt.Or(
        smt.And(pc[n] == PC.Init),
        smt.And(pc[n] == PC.Flag1, flag[n] == True, pc1[n] == PC.Flag2),
    )
)

smt.TransitiveClosure(Spec)

In [ ]:
%%file /tmp/peterson.sv
module peterson(
    input clk,
    input pid,
    output reg flag[2],
    output reg pc[2]
)
@always @(posedge clk) begin
    switch pc[pid]
        PC.Init: begin
            flag[pid] <= 0;
            pc[pid] <= PC.Flag1;
        end
        PC.Flag1: begin
            flag[pid] <= 1;
            pc[pid] <= PC.Flag2;
        end
        PC.Flag2: begin
            if (flag[1 - pid] == 0) begin
                pc[pid] <= PC.Crit;
            end else begin
                pc[pid] <= PC.Flag2; // wait
            end
        end
        PC.Crit: begin
            // Critical section logic here
            pc[pid] <= PC.Exit;
        end
        PC.Exit: begin
            flag[pid] <= 0;
            pc[pid] <= PC.Init;
        end

        assert property !(pc[0] == PC.Crit && pc[1] == PC.Crit);
endmodule

In [ ]:

# 
def lock1(pid, flag):
    assert pid == 0 or pid == 1
    while True:
        yield "a0"
        flag[pid] = True
        yield "a1"
        flag[1 - pid] = False
        yield "a2"

        yield "a3"
        ncrit += 1
        yield "a4"
        assert ncrit == 1  # critical section
        yield "a5"
        ncrit -= 1

        flag[pid] = False
# use hypothesis?
def search():
    while True:
        flag = [False, False]
        procs = [lock1(0, flag), lock1(1, flag)]
        trace = []
        for i in range(10): # 10 steps
            try:
                trace.append(flags.copy(), next(random.choice(procs)))
        






In [ ]:
@dataclass
class State:
    flag: Tuple[bool,bool] = (False, False)
    ncrit: int = 0

def lock(pid):
    assert pid == 0 or pid == 1
    while True:
        yield "a0"; st.flag[pid] = True
        yield "a1"; st.flag[1 - pid] = False 
        yield "a3": st.ncrit += 1
        yield "a4": assert st.ncrit == 1  # critical section
        yield "a5": st.ncrit -= 1
        yield "a6": st.flag[pid] = False

@given(st.lists(st.sampled_from([0,1]), min_size=1, max_size=60))
def test_protocol(sched):
    st = State()
    procs = [lock(pid) for pid in range(2)]
    trace = []
    for step in sched:
        trace.append((step, next(procs[step])))


In [ ]:
def t0(pid, st):
    return st
def t1(pid, st):
    return st
def t2(pid, st):

{
    "t0": lambda pid, st: st,
    "t1": lambda pid, st: st,
    "t2": lambda pid, st: st,
}

proc = [t0,t1,t2,t3]


Spacer. datalog. prolog

Why is a one sided reachablity ok now?

In [ ]:
smt.QForAll([flags, pc], 
       , step()
)

def horn(vs, head, *bodies):
    return [
        smt.ForAll(vs, smt.Implies(smt.And(body), head))
        for body in bodies
    ]
def horn([flag, pc], spec(flags, pc), 
         smt.And(pc[0] == PC.Init, spec() )
         smt.And(pc[0] == PC.Flag1, flag[0] == True, pc1[0] == PC.Flag2) 
         ,
         )

Memory models: Starting point is PC is totally random.
Starting point is everyone has their own copy of memory and PC is random? Or that everyone else's PC is random.

https://github.com/heidihoward/distributed-consensus-reading-list?tab=readme-ov-file#correctness-of-consensus-algorithms

https://github.com/PirateshipOrg/pirateship-tla wha?


# Chapter 3 Concurrent objects
realted?
https://en.wikipedia.org/wiki/Input/output_automaton
https://en.wikipedia.org/wiki/Actor_model


A strange formal model.
Objects, times.
Why is the method invocation vs completion time the thing?
A single global timeline?

https://www.youtube.com/watch?v=nrUszqrlvi8  Concurrent Objects - The Art of Multiprocessor Programming - Part 1

squential consystency
linearizability
queisecnet consistency

histories

FIFO
q.deq()
pointer to head and tail of circular

no locks
one queue thread, one deque

safety and liverness properties
state. has fields.
set of methods.

DeclareSort("queue")
algerbaic spec?
deq(enq(x, emp)) = 
enq_end(enq_start(x))

methods take time

https://dl.acm.org/doi/10.1145/78969.78972 Linearizability: a correctness condition for concurrent objects

https://surfingcomplexity.blog/2024/09/22/linearizability-refinement-prophecy/
https://github.com/lorin/tla-linearizability
https://github.com/lorin/tla-prophecy/blob/master/LinearizableQueue.tla




https://lamport.azurewebsites.net/pubs/pubs.html#simple prophecy made simple - merz and lamport
refinement mappings. proj(impl) = specstate
https://lamport.azurewebsites.net/tla/hiding-and-refinement.pdf rerad before auxiliary paper?
Auxilialty variables in TLA https://lamport.azurewebsites.net/pubs/auxiliary.pdf
remenmber full history vs remember only largest smnallest.
(∃∃ y : Spec1) ≡ (∃∃ min, max : Spec2)

"A history variable remembers the past.
It is needed to find a refinement mapping to show that Spec1 implements Spec2
when Spec2 remembers previous events longer than it has to. A prophecy variable is one that predicts the future. It is needed to find a refinement mapping
to show that Spec1 implements Spec2 when Spec2 makes decisions before it has
to."


Refinement and model order reduction. renormalization
https://faculty.kfupm.edu.sa/phys/imnasser/Phase_transition/Maris_Kadanoff.pdf
exact partition functions via SDD?
Hmm. Maybe it could stay auto differentiable? Tree decompositions. Domain decompositions.
Transform variables to coarse grain votes and prune?
Compress the sdd somehow and track loss of accuracy?
I guess like, pick an add structure and fit with respect to it.
But then you could fix the params and optimize over new structures too. Wasn't declan doing something like this that I was skeptical of. https://declanoller.github.io/2019-01-25-beating-openai-games-with-neuroevolution-agents-pretty-neat "neuroevoluation" NEAT https://en.wikipedia.org/wiki/Neuroevolution_of_augmenting_topologies

e^{beta F} = sum e^{ H}. Effective energy while coarse grainiing.
The sdd variable ordering tree should probably map to some spatial partitioning.
A method to describe `(Bool...) -> R` which if `Bool(pos) -> R` that's a functional on a field.
I guess in my ILP for ising thing, I discussed finding a minimum state in the presence of sources. I had a whole CSP ~ something.
Topological fields ~ frustration globality something something. Spin Ice. Dimer models.


"
Something that tickles some part of my brain is the relationship of all this to dynamical systems, controls systems, classical mechanics etc. Time being of essence makes sense. The distinction between internal states and observations is a common on in kalman filters for example. Invariants (energy, momentum) are a nice method to say something about a dynamical system. Termination measures and lyapunov functions are pretty similar things. know lamport took cues from physics for the clock ideas and a focus on invariants. Something about prophecy variables reminds me of https://en.wikipedia.org/wiki/Green%27s_function#Advanced_and_retarded_Green's_functions a non intuitive variety of green's function for which the influence propagates backwards in time. The more intuitive thing to propagate backwards is kalman filter like stuff, where you use future observations to infer a past state rather than integrate applied forces/control inputs to propagate states forward, but I think it's similar math.
This feels like one of those things that is hard to do anything with except pass over in silence
https://en.wikipedia.org/wiki/Observability
There is something about kalman filters or https://en.wikipedia.org/wiki/Alpha_beta_filter where you store only a little bit of state, whereas in principle you might store the entire history you've seen and run a big honking online opitmization/fit over the entire history
The filtering dual of an mpc https://en.wikipedia.org/wiki/Model_predictive_control
https://en.wikipedia.org/wiki/Moving_horizon_estimation
Refinement kind of reminds me of https://en.wikipedia.org/wiki/Model_order_reduction
"


prophecy - wasn't this part of borrowing tricks in https://www.riec.tohoku.ac.jp/~unno/papers/pldi2025.pdf creusot aeneas
Something is tickling in my brain about green's functions. Advanced / ret / feynman prescription
https://dl.acm.org/doi/10.1145/3371113  The future is ours: prophecy variables in separation logic

refinement of nondet container to fifo or lifo. for every fifo we can describe a container (set). For every container there is more than one fifo quee. We've forgotten something. The set is the list quotiented. A canonical one. Or a sorted list as set. `canon(fifo) = set`. If canon12(state1) = state2 and canon21(state2) = state1 we have a equivalence (?)


Transfer function/ Propagator vs Requirement function (where must this thing be in order to receive a force and end up there (at 0). Observation function? Inference function? )


Specifying Systems uses an example of a linearizable memory in Section 5.3

https://surfingcomplexity.blog/2023/12/31/consistency/

https://muratbuffalo.blogspot.com/2024/08/linearizability-correctness-condition.html failing responses - differerentiated histories trick https://arxiv.org/pdf/1611.00580

https://surfingcomplexity.blog/2024/10/28/serializability-and-tla/

https://davecturner.github.io/2018/02/12/tla-in-isabelle.html

"Consistency properties" https://jepsen.io/consistency/models https://jepsen.io/consistency/models/linearizable
"isolation properties"

https://arxiv.org/pdf/1512.00168  Consistency in Non-Transactional Distributed Storage Systems

There's something analytic feeling about this method interval model. We want to be able to treat intervals as if they were points = linearizable.
Is there a model of the formula   xstart <= x <= xend. A model is a refinement of a kind. Adjunction between syntax and semantics
rays are bad. etendue bundles are good. classical to quantum transition etendue ~ hbar.
volumes not points.

how to distinguish acceptable and unaccpetable siutations.



In [ ]:
Event = kd.Inductive("Event")
Event.declare("enq", ())
Event.declare("deq")

Event = kd.datatype("type Event = Enq(int, int) | Deq(int, int)")
SeqSort(Event)

In [ ]:
%%file /tmp/fifo.java



In [ ]:
import kdrag.theories.logic.temporal as tla

def BigExists(xs, P):
    return smt.Exists(xs, P)
def LittleExists(xs, P): # TExists
    return smt.Lambda([t], smt.Exists([x], P(t)))



What is an interface for a logic?


In [ ]:
from typing import Protocol

class Logic(Protocol):
    def Prop(self, name: str) -> smt.Expr: ...
    and_ : smt.FuncDeclRef
    or_ : smt.FuncDeclRef
    not_ : smt.FuncDeclRef
    implies : smt.FuncDeclRef # Prop -> Prop -> Prop
    valid : smt.FuncDeclRef # Prop -> Bool. A Set?

# interesting...
tele = pdefine("andi", [(p, valid), (q,valid)], valid,
                        and_(p, q), by=[])






### Prophecy


Notes on Prophecy Variables
---------------------------
A system's behavior is a sequence of states
Not all states are always visible; states can be internal. So a behavior is really a sequence of pairs of states, a visible state and an internal state.
A specification should describe only visible state.
A specification should not distinguish between behaviors that differ only with respect to internal states.
A change solely in in internal state causes a "stutter" in the behavior that results from attending to only the visible states; a stutter is a repetition of a visible state.
Let Γs be the set of behaviors that are like s, differing only with respect to stuttering - adding and deleting finite runs of repeated state.
A specification that permits s should permit anything in Γs.[^1]
A low level specification L implements another higher level specification H if they're talking about the same kind of visible state (but maybe specify systems with different internal state. E.g. a algorithm that involves an internal counter vs a hardware implementation of that algorithm), and for every sequence
(v₁,i₁), (v₂,i₂), …
that L allows, there's a corresponding behavior
(v₁,i'₁), (v₂,i'₂), …
with the same visible behavior (up to stuttering), allowed by H. (so if you throw away the internal state on both sides, then closing the behaviors allowed by H under stuttering will include all the behaviors allowed by L). One nice way to prove implementation is to find a refinement mapping f such that if
(v₁,i₁), (v₂,i₂), …
is allowed by L, then
f(v₁,i₁), f(v₂,i₂), …
is allowed by H. Even if L implements H though, the existence of a mapping like this isn't guaranteed.
A refinement mapping might not exist, first of all, because the internal state of H might be more fine-grained than L in a way that it unrecoverable from the state of L. For example, if the visible state is the low order bit, and H is a three bit clock, then H could be implemented by an L with no internal state, and just alternating 1,0 for visible state.
The fix for this sort of situation is to add history variables to L, which basically record what's already happened to L, but don't interact otherwise, so in the case above, you might add a counter n to L's state so that when you need to map an L state to an H state, you can read the value of H's counter off of L's history variable.
You might also have non-determinism that happens at different times, leading to a different kind of non-recoverability. For example, L might pick a random number at each tick of the clock, throwing the first away and displaying the rest, while H might pick one, and display the pick on the next round. So L's behaviors would be like:
    (•,()), (1,()), (50,()), (42,()) …
and H's would be like:
    (•,1), (1,50), (50,42), (42,75) …
In this case, you can't produce a refinement map to recover a corresponding state of H from a state of L, even though they both implement each other. And you can't figure out that the last state in the sequence should be (42,75), no matter how much additional historical information you pack into (42,()). In this case you need to enrich the behaviors of L with prophecy variables. A prophecy variable decides, in advance, what non-deterministic choices L is going to make. In the example above, H is just L plus a prophecy variable.
There's also an extra move where you pack extra stuttering into prophecy variable changes, by not just adding them alongside the existing behaviors of L but by getting the behaviors of the new system by also replacing individual states of L with runs of state in which the L part doesn't change but the prophecy variable bounces around. So prophecy variables can be abused to take care of the scenario where H "runs slower" than L, for example if the visible state is the hour hand of a clock, and H includes minute and second hands, while L includes just the minute hand.
If you define everything *way* more precisely, then the big theorem is, I think, that, subject to reasonable constraints on H, L, if L implements H, then there's an L' obtained from L by adding a prophecy and a history variable such that there's a refinement mapping from L' to H.
[^1]: Such a specification will always be the conjunction of a safety and a liveness property!
If you think of the sequences as topologized like Baire space, where basic clopens are Sᵨ where ρ is a finite sequence, then  a safety property is a closed set, since its complement is defined to be sequences that have certain initial segments, and a liveness property is a dense set, since a liveness property can always happen, so no matter what prefix you chose, some element of the liveness property lives in the corresponding basic open. So actually any set S of behaviors at all is the conjunction of a safety and a liveness property: take the topological closure C of S. The consider the union U of S and the complement of C. I claim U is dense: consider any nonempty open set O. If O does not intersect S, then O's complement contains S, and is closed, and therefore contains C. But then O lies in (and intersects) the complement of C. However U intersect C = S, so S is the intersection of a closed and a dense set

# Chapter 4 Foundations of Shared Memory

# Rust Atomics
https://marabos.nl/atomics/

A processort

